# Data cleaning and handling

In [56]:
#ICTRP Search: "covid-19" or "novel coronavirus" or "2019-ncov" or "covid19" or "sars-cov-2"
import xmltodict
import pandas as pd
import numpy as np
from datetime import date
import re
import unicodedata
from collections import Counter

#POINT THIS TO THE UPDATED XML - deprecated as ICTRP is down.

#with open('ICTRP-Results_18Mar2020.xml', 'rb') as f:
#    xml = xmltodict.parse(f, dict_constructor=dict)

#df = pd.DataFrame(xml['Trials_downloaded_from_ICTRP']['Trial'])

#This now takes the CSV posted by the ICTRP as an input from here: https://www.who.int/ictrp/en/

df = pd.read_excel('this_weeks_data/COVID19-1528-trials_22Apr2020.xlsx', dtype={'Phase': str})

#UPDATE THESE WITH EACH RUN
prior_extract_date = date(2020,4,15)
this_extract_date = date(2020,4,22)

def fix_dates(x):
    if isinstance(x, str):
        return x
    else:
        return x.date()

In [57]:
df['Date enrollement'] = pd.to_datetime(df['Date enrollement'], errors='coerce')

In [58]:
#Extracting target enrollment
size = df['Target size'].tolist()

extracted_size = []
for s in size:
    if not s or isinstance(s,float):
        extracted_size.append('Not Available')
    elif isinstance(s,int):
        extracted_size.append(s)
    elif isinstance(s,str):
        digits = []
        nums = re.findall(r':\d{1,10};',s)
        for n in nums:
            digits.append(int(n.replace(':','').replace(';','')))
        extracted_size.append(sum(digits))
    else:
        print(type(s))

df['target_enrollment'] = extracted_size

#Creating retrospective registration
df['retrospective_registration'] = np.where(df['Date registration'] > df['Date enrollement'], True, False)
df['Date enrollement'] = df['Date enrollement'].fillna('No Date Available').apply(fix_dates)

In [59]:
#Taking only what we need right now

cols = ['TrialID', 'Source Register', 'Date registration', 'Date enrollement', 'retrospective_registration', 
        'Primary sponsor', 'Recruitment Status', 'Phase', 'Study type', 'Countries', 'Public title', 
        'Intervention', 'target_enrollment', 'web address', 'results yes no', 'results url link', 
        'Last Refreshed on']

df_cond = df[cols].reset_index(drop=True)

#renaming columns to match old format so I don't have to change them throughout
df_cond.columns = ['TrialID', 'Source_Register', 'Date_registration', 'Date_enrollement', 
                   'retrospective_registration', 'Primary_sponsor', 'Recruitment_Status', 'Phase', 'Study_type', 
                   'Countries', 'Public_title', 'Intervention', 'target_enrollment', 'web_address', 
                   'has_results', 'results_url_link', 'Last_Refreshed_on']

print(f'The ICTRP shows {len(df_cond)} trials as of {this_extract_date}')

The ICTRP shows 1528 trials as of 2020-04-22


In [60]:
#POINT THIS TO LAST WEEK'S PROCESSED DATA
last_weeks_trials = pd.read_csv('last_weeks_data/trial_list_2020-04-15.csv')

In [61]:
#Joining in the 'first_seen' field
df_cond = df_cond.merge(last_weeks_trials[['trialid', 'first_seen']], left_on = 'TrialID', right_on = 'trialid', 
                        how='left').drop('trialid', axis=1)

#Adding the `first_seen` field to new trials
df_cond['first_seen'] = pd.to_datetime(df_cond['first_seen'].fillna(this_extract_date))

In [62]:
#Check for which registries we are dealing with:
df_cond.Source_Register.value_counts()

ChiCTR                             617
ClinicalTrials.gov                 614
IRCT                               111
EU Clinical Trials Register         72
ANZCTR                              25
German Clinical Trials Register     24
ISRCTN                              16
Netherlands Trial Register          13
JPRN                                13
CTRI                                 7
TCTR                                 6
REBEC                                5
PACTR                                3
REPEC                                1
RPCEC                                1
LBCTR                                1
Name: Source_Register, dtype: int64

When working with data straight from XML we need to do some tedious tidying up of dates because of different formats. They do not parse correctly by default in Pandas. They are standardized, however, in the ICTRP spreadsheet so I have removed this code for now. It is archived in old commits to the GitHub repo for future refrence if needed.

In [63]:
#lets get rid of trials from before 2020 for now

pre_2020 = len(df_cond[df_cond['Date_registration'] < pd.Timestamp(2020,1,1)])

print(f'Excluded {pre_2020} trials from before 2020')

df_cond_rec = df_cond[df_cond['Date_registration'] >= pd.Timestamp(2020,1,1)].reset_index(drop=True)

print(f'{len(df_cond_rec)} trials remain')

Excluded 9 trials from before 2020
1520 trials remain


This code removes trials that indicate that they never started. This is done on the Chinese registry through specific language in the Title. Trials from ClinicalTrials.gov are indicated by the `Withdrawn` trial status. 

This will be expanded moving forward to account for the unique terminology used by other registries as necessary moving forward.

In [64]:
#Removing cancelled/withdrawn trials for what registries we have to date

cancelled_trials = len(df_cond_rec[(df_cond_rec['Public_title'].str.contains('Cancelled')) | 
                                   (df_cond_rec['Public_title'].str.contains('Retracted due to')) | 
                                   (df_cond_rec['Recruitment_Status'] == "Withdrawn")])

print(f'Excluded {cancelled_trials} cancelled trials with no enrollment')

#Now lets get rid of trials we know don't belong from manual review, reason catalogued below
#NCT04226157 Non-COVID trial delayed because of COVID and put this in the title
#NCT04246242 This trial registration doesn't exist anymore
#NCT04337320 is monitoring of complications from a device in the context of COVID but has nothing
#to do with COVID monitoring or treatment.
#NCT03680274 simply allows inclusion of COVID pts but has nothing to do with COVID
#JPRN-UMIN000040188 is a systematic review, not an individual study

exclude = ['NCT04226157', 'NCT04246242', 'NCT04337320', 'NCT03680274', 'JPRN-UMIN000040188']

print(f'Excluded {len(exclude)} non-COVID trials screened through manual review')

df_cond_nc = df_cond_rec[~((df_cond_rec['Public_title'].str.contains('Cancelled')) | 
                           (df_cond_rec['Public_title'].str.contains('Retracted due to'))) & 
                         ~(df_cond_rec['Recruitment_Status'] == "Withdrawn") &
                         ~(df_cond_rec['TrialID'].isin(exclude))].reset_index(drop=True)

print(f'{len(df_cond_nc)} trials remain')

Excluded 44 cancelled trials with no enrollment
Excluded 5 non-COVID trials screened through manual review
1473 trials remain


As a general rule, simply for quality and ease of use, we will default to a ClinicalTrials.gov record over another type of registration in instances of cross-registration. The ICTRP alerts users to trials with known cross-registrations in the "Bridge" field of their dataset and only lists 1 registration per trial (but does not tell you the cross-registered trial IDs). We can manually check and catalogue these. However we will want to replace some of these when the "Parent" registry is the EUCTR entry. The first step is to remove the EUCTR entries, then we will add the ClinicalTrials.gov (or another) version of the registry entry  back into the dataset when we append known trials. We will then join in a new column listing the known cross-registered trial ids.

In [65]:
c_reg = pd.read_excel('manual_data.xlsx', sheet_name = 'cross registrations')
replace_ids = c_reg.id_to_replace.tolist()

replaced = len(df_cond_nc[df_cond_nc.TrialID.isin(replace_ids)])
print(f'{replaced} known cross registrations will be replaced')

df_cond_nc = df_cond_nc[~(df_cond_nc.TrialID.isin(replace_ids))].reset_index(drop=True)

16 known cross registrations will be replaced


In [66]:
#Function for checking changes

def trial_diffs(new=True):
    df = df_cond_nc.merge(last_weeks_trials['trialid'], left_on = 'TrialID', right_on = 'trialid', how='outer', indicator=True)
    if new:
        new = df[(df['_merge'] == 'left_only')]
        return new['TrialID'].tolist()
    else:
        dropped = df[(df['_merge'] == 'right_only')]
        return dropped['trialid'].tolist()

In [67]:
additions = pd.read_excel('manual_data.xlsx', sheet_name = 'additional_trials')

print(f'There are approximately {len(trial_diffs(new=True))} new trials')

added = additions.TrialID.tolist()

print(f'The following trials were removed since the last time and were manually checked to confirm:')
print(list(set(trial_diffs(new=False)) - set(added) - set(replace_ids)))

There are approximately 383 new trials
The following trials were removed since the last time and were manually checked to confirm:
['ChiCTR2000030300']


Now we just need to take a quick look at trials that came and went since the last update. We can add in any additional trials that we know about that are not accounted for in the ICTRP database.

In [68]:
#These are trials we know about that are not showing up in the ICTRP data pull or are re-added as cross-registrations

print(f'An additional {len(additions)} known trials, or preferred cross registrations were added to the data')

print(added)

df_cond_all = df_cond_nc.append(additions)

print(f'The final dataset is {len(df_cond_all)} trials')

An additional 18 known trials, or preferred cross registrations were added to the data
['NCT04299152', 'NCT04307459', 'NCT04290780', 'NCT04303299', 'NCT04321369', 'NCT04323527', 'NCT04333914', 'NCT04328285', 'NCT04341207', 'NCT04339712', 'NCT04341038', 'NCT04335786', 'NCT04345289', 'NCT04359953', 'NCT04334850', 'NCT04344288', 'NCT04350580', 'NCT04344041']
The final dataset is 1475 trials


In [69]:
#finally, add cross-registration field

df_cond_all = df_cond_nc.merge(c_reg[['trial_id_keep', 'additional_ids']].drop_duplicates(), 
                              left_on='TrialID', 
                              right_on='trial_id_keep', 
                              how='left').drop('trial_id_keep', axis=1).rename(columns=
                                                                               {'additional_ids':
                                                                                'cross_registrations'}
                                                                              ).reset_index(drop=True)

df_cond_all['cross_registrations'] = df_cond_all['cross_registrations'].fillna('None')

Normalisation and data cleaning of all fields. This will be expanded each update as more trials get added and more registries start to add trials with their own idiosyncratic data categories. 

In [70]:
#A small function to help quickly check the contents of various columns

def check_fields(field):
    return df_cond_all[field].unique()

#Check fields for new unique values that require normalisation
#for x in check_fields('Countries'):
#    print(x)

In [71]:
#Data cleaning various fields. 
#One important thing we have to do is make sure there are no nulls or else the data won't properly load onto the website

#semi-colons in the intervention field mess with CSV
df_cond_all['Intervention'] = df_cond_all['Intervention'].str.replace(';', '')

#Study Type
obv_replace = ['Observational [Patient Registry]', 'observational']
int_replace = ['interventional', 'Interventional clinical trial of medicinal product', 'Treatment', 
               'INTERVENTIONAL', 'Intervention']
hs_replace = ['Health services reaserch', 'Health Services reaserch', 'Health Services Research']

df_cond_all['Study_type'] = df_cond_all['Study_type'].str.replace(' study', '')
df_cond_all['Study_type'] = df_cond_all['Study_type'].replace(obv_replace, 'Observational')
df_cond_all['Study_type'] = df_cond_all['Study_type'].replace(int_replace, 'Interventional')
df_cond_all['Study_type'] = df_cond_all['Study_type'].replace('Epidemilogical research', 'Epidemiological research')
df_cond_all['Study_type'] = df_cond_all['Study_type'].replace(hs_replace, 'Health services research')
df_cond_all['Study_type'] = df_cond_all['Study_type'].replace('Others,meta-analysis etc', 'Other')

#phase
df_cond_all['Phase'] = df_cond_all['Phase'].fillna('Not Applicable')
phase_fixes = {'0':'Not Applicable', '1':'Phase 1', '2':'Phase 2', '3':'Phase 3', '4':'Phase 4', 
               '1-2':'Phase 1/Phase 2', 'Retrospective study':'Not Applicable', 
               'Not applicable':'Not Applicable', 'Early Phase 1':'Phase 1',
               'New Treatment Measure Clinical Study':'Not Applicable', 'Not selected': 'Not Applicable',
               '2020-02-01 00:00:00':'Phase 1/Phase 2', 'Phase II/III':'Phase 2/Phase 3',
               '2020-03-02 00:00:00':'Phase 2/Phase 3', 'Phase III':'Phase 3',
               'Phase I/II':'Phase 1/Phase 2', 'Phase 0': 'Not Applicable',
               'Phase 1 / Phase 2': 'Phase 1/Phase 2', 'I': 'Phase 1', 'II':'Phase 2', 
               'II-III':'Phase 2/Phase 3', 'IV': 'Phase 4', 'Phase-3': 'Phase 3', 
               'Diagnostic New Technique Clincal Study': 'Not Applicable',
               'Human pharmacology (Phase I): no\nTherapeutic exploratory (Phase II): yes\nTherapeutic confirmatory - (Phase III): no\nTherapeutic use (Phase IV): no\n': 'Phase 2',
               'Human pharmacology (Phase I): no\nTherapeutic exploratory (Phase II): no\nTherapeutic confirmatory - (Phase III): yes\nTherapeutic use (Phase IV): no\n': 'Phase 3',
               'Human pharmacology (Phase I): no\nTherapeutic exploratory (Phase II): no\nTherapeutic confirmatory - (Phase III): no\nTherapeutic use (Phase IV): yes\n': 'Phase 4',
               'Human pharmacology (Phase I): no\nTherapeutic exploratory (Phase II): no\nTherapeutic confirmatory - (Phase III): yes\nTherapeutic use (Phase IV): yes\n': 'Phase 3/Phase 4',
               'Human pharmacology (Phase I): no\nTherapeutic exploratory (Phase II): yes\nTherapeutic confirmatory - (Phase III): yes\nTherapeutic use (Phase IV): no\n': 'Phase 2/Phase 3'
              }
df_cond_all['Phase'] = df_cond_all['Phase'].replace(phase_fixes)

#Fixing Observational studies incorrectly given a Phase in ICTRP data
m = ((df_cond_all.Phase.str.contains('Phase')) & (df_cond_all.Study_type == 'Observational'))
df_cond_all['Phase'] = df_cond_all.Phase.where(~m, 'Not Applicable')

#Recruitment Status
df_cond_all['Recruitment_Status'] = df_cond_all['Recruitment_Status'].replace('Not recruiting', 'Not Recruiting')
df_cond_all['Recruitment_Status'] = df_cond_all['Recruitment_Status'].fillna('No Status Given')

#Get rid of messy accents

def norm_names(x):
    normed = unicodedata.normalize('NFKD', str(x)).encode('ASCII', 'ignore').decode()
    return normed 

df_cond_all['Primary_sponsor'] = df_cond_all.Primary_sponsor.apply(norm_names)
df_cond_all['Primary_sponsor'] = df_cond_all['Primary_sponsor'].replace('NA', 'No Sponsor Name Given')
df_cond_all['Primary_sponsor'] = df_cond_all['Primary_sponsor'].replace('nan', 'No Sponsor Name Given')

In [72]:
#Countries
df_cond_all['Countries'] = df_cond_all['Countries'].fillna('No Country Given').replace('??', 'No Country Given')

china_corr = ['Chian', 'China?', 'Chinese', 'Wuhan', 'Chinaese', 'china', 'Taiwan, Province Of China', 
              "The People's Republic of China"]

country_values = df_cond_all['Countries'].tolist()

new_list = []

for c in country_values:
    country_list = []
    if isinstance(c, float):
        country_list.append('No Sponsor Name Given')
    elif c == 'No Sponsor Name Given':
        country_list.append('No Sponsor Name Given')
    elif c in china_corr:
        country_list.append('China')
    elif c in ['Iran (Islamic Republic of)', 'Iran, Islamic Republic of']:
        country_list.append('Iran')
    elif c in ['Viet nam', 'Viet Nam']:
        country_list.append('Vietnam')
    elif c in ['Korea, Republic of', 'Korea, Republic Of', 'KOREA'] :
        country_list.append('South Korea')
    elif c in ['USA', 'United States of America']:
        country_list.append('United States')
    elif c == 'Japan,Asia(except Japan),Australia,Europe':
        country_list = ['Japan', 'Australia', 'Asia', 'Europe']
    elif c == 'The Netherlands':
        country_list.append('Netherlands')
    elif c == 'England':
        country_list.append('United Kingdom')
    elif c == 'Japan,North America':
        country_list = ['Japan', 'North America']
    elif ';' in c:
        c_list = c.split(';')
        unique_values = list(set(c_list))
        for v in unique_values:
            if v in china_corr:
                country_list.append('China')
            elif v in ['Iran (Islamic Republic of)', 'Iran, Islamic Republic of']:
                country_list.append('Iran')
            elif v in ['Korea, Republic of', 'Korea, Republic Of', 'KOREA']:
                country_list.append('South Korea')
            elif v in ['Viet nam', 'Viet Nam']:
                country_list.append('Vietnam')
            elif v in ['USA', 'United States of America']:
                country_list.append('United States')
            elif v == 'The Netherlands':
                country_list.append('Netherlands')
            elif v == 'England':
                country_list.append('United Kingdom')
            else:
                country_list.append(v)
    else:
        country_list.append(c)
    new_list.append(', '.join(country_list))

df_cond_all['Countries'] = new_list

Last space for manual intervention. This will include manual normalisation of new names, any updates to the normalisation schedule from the last update, and updating manually-coded intervention type data.

In [73]:
#Normalizing sponsor names
#Run this cell, updating the spon_norm csv you are loading after manual adjusting
#until you get the 'All sponsor names normalized' to print

spon_norm = pd.read_excel('manual_data.xlsx', sheet_name = 'sponsor')

df_cond_norm = df_cond_all.merge(spon_norm, left_on = 'Primary_sponsor', right_on ='unique_spon_names', how='left')
df_cond_norm = df_cond_norm.drop('unique_spon_names', axis=1)

new_unique_spon_names = (df_cond_norm[df_cond_norm['normed_spon_names'].isna()][['Primary_sponsor', 'TrialID']]
                        .groupby('Primary_sponsor').count())

if len(new_unique_spon_names) > 0:
    new_unique_spon_names.to_csv('to_norm.csv')
    print('Update the normalisation schedule and rerun')
else:
    print('All sponsor names normalized')

All sponsor names normalized


In [74]:
#Integrating intervention type data
#Once again, run to bring in the old int-type data, islolate the new ones, update, and rerun until
#producing the all-clear message

int_type = pd.read_excel('manual_data.xlsx', sheet_name = 'intervention')
df_cond_int = df_cond_norm.merge(int_type[['trial_id', 'study_category',
                                           'intervention', 'intervention_list']], 
                                 left_on = 'TrialID', right_on = 'trial_id', how='left')

df_cond_int = df_cond_int.drop('trial_id', axis=1)

new_int_trials = df_cond_int[(df_cond_int['study_category'].isna()) | (df_cond_int['intervention'].isna())]

if len(new_int_trials) > 0:
    new_int_trials[['TrialID', 'Public_title', 'Intervention', 'study_category', 
                    'intervention', 'intervention_list']].to_csv('int_to_assess.csv')
    print('Update the intervention type assessments and rerun')
else:
    print('All intervention types matched')
    df_cond_int = df_cond_int.drop('Intervention', axis=1).reset_index(drop=True)

All intervention types matched


In [75]:
#Can use this cell to output counts of values from columns

treatments = df_cond_int[df_cond_int.study_category == 'Drug']['study_category'].tolist()
countries = df_cond_int.Countries.to_list()

def var_counts(var_list, split_char, lower=False):
    final_list = []
    for v in var_list:
        t_list = v.split(split_char)
        for l in t_list:
            if lower:
                final_list.append(l.lower().strip())
            else:
                final_list.append(l.strip())
    return Counter(final_list)

In [76]:
comp_dates = pd.read_excel('manual_data.xlsx', sheet_name = 'Completion Dates')
df_comp_dates = df_cond_int.merge(comp_dates, 
                                  left_on='TrialID', right_on='trialid', 
                                  how='left', indicator=True).drop('trialid', axis=1)

print(df_comp_dates[df_comp_dates['_merge'] == 'left_only'].TrialID.to_list())

df_comp_dates = df_comp_dates.drop('_merge', axis=1).reset_index(drop=True)
    
df_comp_dates['primary_completion_date'] = (pd.to_datetime(df_comp_dates['primary_completion_date'], 
                                                          errors='coerce', 
                                                          format='%Y-%m-%d')
                                            .fillna('Not Available').apply(fix_dates))

df_comp_dates['full_completion_date'] = (pd.to_datetime(df_comp_dates['full_completion_date'], 
                                                          errors='coerce', 
                                                          format='%Y-%m-%d')
                                            .fillna('Not Available').apply(fix_dates))

[]


In [77]:
#check for any results on ICTRP
ictrp_results = df_comp_dates[(df_comp_dates.has_results.notnull()) | (df_comp_dates.has_results.notnull())]

if len(ictrp_results) > 0:
    print(f'There are {len(ictrp_results)} results to check')
else:
    print('There are no results to check')

#If results cross-check with results already collected in 'manual_data' excel file and add any new trial results.

There are no results to check


In [78]:
results = pd.read_excel('manual_data.xlsx', sheet_name = 'Results')
df_results = df_comp_dates.merge(results, 
                                 left_on='TrialID', 
                                 right_on='trialid', 
                                 how='left').drop('trialid', axis=1)

df_results['results_link'] = df_results['results_link'].fillna('No Results')
df_results['results_type'] = df_results['results_type'].fillna('No Results')

df_results['results_publication_date'] = (pd.to_datetime(df_results['results_publication_date'], 
                                                          errors='coerce', 
                                                          format='%Y-%m-%d')
                                            .fillna('No Results').apply(fix_dates))

In [79]:
#Final organising

col_names = []

for col in list(df_results.columns):
    col_names.append(col.lower())
    
df_results.columns = col_names

reorder = ['trialid', 'source_register', 'date_registration', 'date_enrollement', 'retrospective_registration', 
           'normed_spon_names', 'recruitment_status', 'phase', 'study_type', 'countries', 'public_title', 
           'study_category', 'intervention', 'target_enrollment', 'primary_completion_date', 
           'full_completion_date', 'web_address', 'results_type', 'results_publication_date', 'results_link', 
           'last_refreshed_on', 'first_seen', 'cross_registrations']

df_final = df_results[reorder].reset_index(drop=True).reset_index()

In [80]:
#Checking for any null values
df_final[df_final.isna().any(axis=1)]

,index,trialid,source_register,date_registration,date_enrollement,retrospective_registration,normed_spon_names,recruitment_status,phase,study_type,...,target_enrollment,primary_completion_date,full_completion_date,web_address,results_type,results_publication_date,results_link,last_refreshed_on,first_seen,cross_registrations


In [81]:
#Quick look at the data
df_final.head()

,index,trialid,source_register,date_registration,date_enrollement,retrospective_registration,normed_spon_names,recruitment_status,phase,study_type,...,target_enrollment,primary_completion_date,full_completion_date,web_address,results_type,results_publication_date,results_link,last_refreshed_on,first_seen,cross_registrations
0,0,ChiCTR2000029953,ChiCTR,2020-02-17,2020-02-01,True,Zhongnan Hospital of Wuhan University,Not Recruiting,Not Applicable,Observational,...,400,Not Available,2021-05-01,http://www.chictr.org.cn/showproj.aspx?proj=49217,No Results,No Results,No Results,2020-02-17,2020-08-03,None
1,1,ChiCTR2000029935,ChiCTR,2020-02-16,2020-02-06,True,"Hwa Mei Hospital, University of Chinese Academ...",Recruiting,Not Applicable,Interventional,...,100,Not Available,2021-02-06,http://www.chictr.org.cn/showproj.aspx?proj=49607,No Results,No Results,No Results,2020-02-17,2020-08-03,None
2,2,ChiCTR2000029850,ChiCTR,2020-02-15,2020-02-15,False,The First Affiliated Hospital of Zhejiang Univ...,Recruiting,Not Applicable,Interventional,...,20,Not Available,2022-02-15,http://www.chictr.org.cn/showproj.aspx?proj=49533,No Results,No Results,No Results,2020-02-17,2020-08-03,None
3,3,ChiCTR2000029814,ChiCTR,2020-02-14,2020-02-14,False,Children's Hospital of Fudan University,Recruiting,Not Applicable,Interventional,...,30,Not Available,2020-12-31,http://www.chictr.org.cn/showproj.aspx?proj=49387,No Results,No Results,No Results,2020-02-17,2020-08-03,None
4,4,ChiCTR2000029805,ChiCTR,2020-02-14,2020-02-12,True,Wuhan Jinyintan Hospital (Wuhan Infectious Dis...,Recruiting,Not Applicable,Observational,...,100,Not Available,2021-01-31,http://www.chictr.org.cn/showproj.aspx?proj=49188,No Results,No Results,No Results,2020-02-17,2020-08-03,None


In [82]:
#Export final dataset
df_final.to_csv(f'processed_data_sets/trial_list_{this_extract_date}.csv', index=False)

df_final.to_csv(f'tableau_data/current_data.csv', index=False)

In [83]:
just_results = df_final[df_final.results_type != 'No Results']

results_total = len(just_results)

print(f'There are {results_total} trials with results')

There are 35 trials with results


In [ ]:
#Export json for website
import json
with open("website_data/trials_latest.json", "w") as f:
    json.dump({"data": df_final.astype(str).values.tolist()}, f, indent=2)

with open("website_data/results_latest.json", "w") as f:
    json.dump({"data": just_results.astype(str).values.tolist()}, f, indent=2)    

# Overall Trend in Registered Trials Graph

In [ ]:
just_reg = df_final[['trialid', 'date_registration']].reset_index(drop=True)
#just_reg = mar18[['trialid', 'date_registration']].reset_index(drop=True)
#just_reg['date_registration'] = pd.to_datetime(just_reg['date_registration'], format='%d/%m/%Y')

#catch old registrations that were expanded to include COVID, we can get rid of these for now
just_reg = just_reg[just_reg['date_registration'] >= pd.Timestamp(2020,1,1)].reset_index(drop=True)

In [ ]:
just_reg.index = just_reg['date_registration']


grouped = just_reg.resample('W').count()
cumsum = grouped.cumsum()

In [ ]:
import matplotlib.pyplot as plt

labels = []

for x in list(grouped.index):
    labels.append(str(x.date()))

x_pos = [i for i, _ in enumerate(labels)]

fig, ax = plt.subplots(figsize=(10,5), dpi = 300)

l1 = plt.plot(x_pos, grouped['trialid'], marker = 'o')
l2 = plt.plot(x_pos, cumsum['trialid'], marker = 'o')

for i, j in zip(x_pos[1:], grouped['trialid'].tolist()[1:]):
    ax.annotate(str(j), (i,j), xytext = (i-.1, j-50))

for i, j in zip(x_pos, cumsum['trialid']):
    ax.annotate(str(j), (i,j), xytext = (i-.2, j+25))
    

gr = grouped['trialid'].to_list()
cs = cumsum['trialid'].to_list()

plt.xticks(x_pos, labels, rotation=45, fontsize=8)
plt.ylim(-50,1200)
plt.xlabel('Week Ending Date')
plt.ylabel('Registered Trials')
plt.title('Registered COVID-19 Trials by Week on the ICTRP')
plt.legend(('New Trials', 'Cumulative Trials'), loc=2)
#plt.savefig(f'trial_count_{last_extract_date}.png')
plt.show()

In [ ]:
import plotly.graph_objects as go

labels = []

for x in list(grouped.index):
    labels.append(str(x.date()))

fig = go.Figure()

fig.add_trace(go.Scatter(x=labels[:-1], y=grouped['trialid'][:-1], fill=None, name='New Trials'))

fig.add_trace(go.Scatter(x=labels[:-1], y=cumsum['trialid'][:-1], fill=None, name='Cumulative Trials'))

fig.update_layout(title={'text': 'Registered COVID-19 Trials by Week', 'xanchor': 'center', 'x': 0.5}, 
                  xaxis_title='Week Ending Date',
                  yaxis_title='Registered Trials',
                  legend = dict(x=0, y=1, traceorder='normal', bgcolor='rgba(0,0,0,0)'))



fig.show()
fig.write_html("html_figures/registered trials.html")

In [ ]:
int_types = df_final.study_category.value_counts()
int_types

In [ ]:
treatment_dict = dict(drugs = int_types['Drug'] + int_types['Drug (Chemoprophylaxis)'], 
                      atmp = int_types['ATMP'], 
                      clinical_char = int_types['Clinical Presentation'] + int_types['Diagnostics'] + int_types['Prognosis'],
                      drug_other_combo = int_types['Drug (+ATMP +Other (renal))'] + int_types['Drug (+ATMP)'],
                      supp = int_types['Supplement'],
                      geno = int_types['Genomics'],
                      th = int_types['Telehealth'],
                      tm = int_types[[s.startswith('Traditional Medicine') for s in int_types.index]].sum(),
                      other = (int_types[[s.startswith('Other') for s in int_types.index]].sum() 
                               + int_types['Health System'] + int_types['Procedure'])
                     )

fig = go.Figure(go.Bar(
            x=list(treatment_dict.values()),
            y=['Drugs', 'ATMP', 'Clinical Characteristics', 'Drug + Other Therapy', 'Supplement', 'Genomics', 
               'Telehealth', 'Traditional Medicine', 'Other'],
            orientation='h'))

fig.update_layout(title={'text': 'Intervention Type of Registered Trials', 'xanchor': 'center', 'x': 0.5}, 
                  xaxis_title='Number of Trials')

fig.show()
fig.write_html('html_figures/int_bar.html')